In [3]:
!pip install --quiet qiskit

In [7]:
!pip install --quiet qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 29.8 MB/s eta 0:00:0000:010:01


In [9]:
import random
# Import Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_state_city

In [16]:
def encode_qubits(bits, bases):
    ''' 
        bits - list of bits in str type ('0' or '1')
        bases - list of bases ('X' or 'Z')
        
        job - encodes new qubits by scheme with given bits and bases
        
        predefined scheme:
            Z   X
        0  |0> |+>
        1  |1> |->
        
        return - list of encoded qubits
    '''
    
    # ensure that same amount of bits and bases are received
    assert len(bits) == len(bases)
    
    encoded_qubits = []
    for bit, base in zip(bits, bases):
        # Create new quantum circuit for each qubit
        new_qc = QuantumCircuit(1, 1)
       
        # Apply encoding based on the given scheme
        if bit == '0':
            if base == 'Z':
                # 0 - Z -> |0>
                pass  # Already in |0>
            elif base == 'X':
                # 0 - X -> |+>
                new_qc.h(0)
        elif bit == '1':
            if base == 'Z':
                # 1 - Z -> |1>
                new_qc.x(0)
            elif base == 'X':
                # 1 - X -> |->
                new_qc.x(0)
                new_qc.h(0)
        
        encoded_qubits.append(new_qc)
        
    return encoded_qubits


def measure_qubits(qubits, bases):
    '''
        qubits - received list of qubits to measure
        bases - list of bases ('X' or 'Z') to measure qubits in
        
        job - Measures qubits with given bases
        
        return - list of bits given from performed measurement
    '''
    result_bits = []
    
    simulator = AerSimulator()  # Define simulator once before loop

    for qubit, base in zip(qubits, bases):
        # Create a new circuit to measure the qubit
        new_qc = QuantumCircuit(1, 1)
        new_qc.compose(qubit, inplace=True)  # Add the encoded qubit to the circuit

        # Apply the measurement basis
        if base == 'X':
            new_qc.h(0)  # Apply Hadamard before measurement

        new_qc.measure(0, 0)  # Measure the qubit

        # Transpile the circuit for simulation
        transpiled_qc = transpile(new_qc, simulator)

        # Run and get measurement results
        result = simulator.run(transpiled_qc).result()
        counts = result.get_counts(transpiled_qc)

        # Get the most probable measured bit (either '0' or '1')
        measured_bit = max(counts, key=counts.get)
        result_bits.append(measured_bit)

    return result_bits
      
  


def generate_bits(n):
    ''' n amount of bits generator '''
    return [random.choice(['0', '1']) for i in range(n)]


def generate_bases(n):
    ''' n amount of bases generator '''
    return [random.choice(['Z', 'X']) for i in range(n)]    


def eliminate_differences(bits, indexes):
    ''' 
        new list of bits is created by appending bits from user bits,
        where base indexes are matched. Used in comparing phase to filter out
        agreed bits by matched bases
    '''
    result_key = []
    for i in range(len(bits)):
        if i in indexes:
            result_key.append(bits[i])
    
    return result_key

In [21]:
''' Phase 1: Generate, encode and send '''

# Alice generates 500 random bits and bases
alice_bits = generate_bits(500)
alice_bases = generate_bases(500)

# Encoding qubits by generated bits and bases
encoded_qubits = encode_qubits(alice_bits, alice_bases)

# print data 
print('Alice Bits:')
print(alice_bits)

print('Alice Bases:')
print(alice_bases)

# qubits are sent to bob with quantum channel ...

Alice Bits:
['0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0

In [22]:
''' 
    Phase 3: Bob recieves qubits from Alice, bob does 
    not know yet that qubits are intercepted by Eve
'''

# bob measures recieved qubits with random bases
bob_bases = generate_bases(500)
bob_bits = measure_qubits(encoded_qubits, bob_bases)

# print data 
print('Bob Bases:')
print(bob_bases)

print('Bob Bits:')
print(bob_bits)

Bob Bases:
['X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z'

In [23]:
''' Phase 4: Comparing '''

# alice and bob compare bases on public channel, both eliminates own bits where bases are different.

# determine indexes of matched bases in a list
same_base_indexes = []
for i in range(len(alice_bases)):
    if alice_bases[i] == bob_bases[i]:
        same_base_indexes.append(i)
        
# Alice generates own key by eliminating bits
# encoding with different bases by Alice and Bob
alice_key = eliminate_differences(
    alice_bits,
    same_base_indexes
)

print('Alice key:')
print(alice_key)

# Bob does the same...
bob_key = eliminate_differences(
    bob_bits,
    same_base_indexes
)

print('Bob key:')
print(bob_key)


# if keys are different, qubits are surely intercepted by someone.
# else, secret key is safe to encrypt information with
if alice_key == bob_key:
    print('Key is safe to use!')
    print(f'Key length: {len(alice_key)}') 
else:
    print('\n Key is compromised and is not safe!')

Alice key:
['0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '1', '0', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '0'

In [24]:
import binascii

def encrypt_message(unencrypted_string, key):
    # Convert ascii string to binary string
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    # created the encrypted string using the key
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )
    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    # created the unencrypted string using the key
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    # Convert bitstring into
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string

In [25]:
# Alice encrypts secret message with key that is already distributed.
secret_message = 'Quantum Computing is cool :)'
encrypted_message = encrypt_message(secret_message, alice_key)

print('Alice message:', secret_message)
print('\nEncrypted message:', encrypted_message)


Alice message: Quantum Computing is cool :)

Encrypted message: 00000100001011001000001000100001001001110010101111001110000000101000000000100011011111110010110101000011110000101111000001000100111100110100011100110100110101000110001110010000000101100101010000000110101001011110010011001110


In [26]:
# Bob recieved encrypted message and decrypts it with own key.
decrypted_message = decrypt_message(encrypted_message, bob_key)

print('Message decrypted by Bob:', decrypted_message)

Message decrypted by Bob: Quantum Computing is cool :)
